# Data inspection for rt_trips_DB_2018

As per the Concept Design Extract:
- Each record of this table contains the actual data for one trip.
- If the trip is planned and achieved, the record will contain both the planned and actual data.
- If the tripped is planned and suppressed the record will contain only the planned data and the flag “Suppressed” will be checked.
- If the trips is partially suppressed the flag “Suppressed” will not be checked. The set of the leave
- times suppressed is detailed in table RT_LeaveTimes ( 9.2.3 ).
- Each record could be referred to a fault event by using the field “JustificationID”.

#### Import Necessary Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date

#### Read in trips file

In [4]:
df = pd.read_csv("~/data/Data_Files/rt_trips_DB_2018.txt", delimiter = ';')

In [8]:
df.loc[df['LINEID']== '41D'].shape

(36, 16)

The file is large- 

In [4]:
df.shape

(2182637, 16)

In [5]:
df.columns

Index(['DATASOURCE', 'DAYOFSERVICE', 'TRIPID', 'LINEID', 'ROUTEID',
       'DIRECTION', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR',
       'ACTUALTIME_DEP', 'BASIN', 'TENDERLOT', 'SUPPRESSED', 'JUSTIFICATIONID',
       'LASTUPDATE', 'NOTE'],
      dtype='object')

In [6]:
# remove any whitespaces in the column names
df.columns = df.columns.str.replace(' ', '')

#### Convert features to appropriate datatypes

In [7]:
df.dtypes

DATASOURCE          object
DAYOFSERVICE        object
TRIPID               int64
LINEID              object
ROUTEID             object
DIRECTION            int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR     float64
ACTUALTIME_DEP     float64
BASIN               object
TENDERLOT          float64
SUPPRESSED         float64
JUSTIFICATIONID    float64
LASTUPDATE          object
NOTE                object
dtype: object

__Examing the data we will now convert columns into their appropriate types__

- **Datasource** is categorical as it is a defined as"DB" for Dublin Bus
- **Day of Service** is a date and thus will be converted as such
- **TripID** is categorical as it is a label for trips and thus does make sense to asses in a continous manner
- **LineID  & RouteID** are also categorical as it corresponds to the bus line which is a set number
- **Direction** is categorical as it is either inbound or outbound
- **PlannedTime_Dep, PlannedTime_Arr and ActualTime_Dep, ActualTime_Arr** - continous data representing trip time in seconds
- **Basin, Tenderlot, Suprressed, Justification and Note** - All categorical
- **LastUpdate** - datetime


__Continous Columns will be converted to float32 for memory preservation__

In [8]:
cat_cols = ['DATASOURCE', 'TRIPID', 'LINEID', 'ROUTEID', 'DIRECTION','BASIN', 'TENDERLOT', 'SUPPRESSED', 'JUSTIFICATIONID','NOTE']
cont_cols = ['PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR',
       'ACTUALTIME_DEP']
for column in cont_cols:
    df[column]= df[column].astype('float32')
    
for column in cat_cols:   
     df[column]= df[column].astype('category')
        
date_time=['DAYOFSERVICE','LASTUPDATE']  
df[date_time]=df[date_time].apply(pd.to_datetime)

In [9]:
df.dtypes

DATASOURCE               category
DAYOFSERVICE       datetime64[ns]
TRIPID                   category
LINEID                   category
ROUTEID                  category
DIRECTION                category
PLANNEDTIME_ARR           float32
PLANNEDTIME_DEP           float32
ACTUALTIME_ARR            float32
ACTUALTIME_DEP            float32
BASIN                    category
TENDERLOT                category
SUPPRESSED               category
JUSTIFICATIONID          category
LASTUPDATE         datetime64[ns]
NOTE                     category
dtype: object

### Check for duplicates

In [10]:
#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', df.duplicated().sum())

# Check for duplicate rows. 
# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', df[df.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


**No Duplicates found**

## Continous Statistics will be evaluated below

In [11]:
df.describe(datetime_is_numeric=True).T

,count,mean,min,25%,50%,75%,max,std
DAYOFSERVICE,2182637,2018-06-28 17:25:38.969786880,2018-01-01 00:00:00,2018-03-31 00:00:00,2018-06-28 00:00:00,2018-09-26 00:00:00,2018-12-31 00:00:00,NaN
PLANNEDTIME_ARR,2182637.0,55078.746094,18203.0,39683.0,55276.0,69006.0,91680.0,17424.822266
PLANNEDTIME_DEP,2182637.0,51533.277344,16200.0,36000.0,51600.0,65160.0,87600.0,17571.636719
ACTUALTIME_ARR,2045430.0,55062.398438,17546.0,39668.0,55189.0,69019.0,94378.0,17338.699219
ACTUALTIME_DEP,2018086.0,51706.125,15974.0,36518.0,51618.0,65398.0,97177.0,17526.949219
LASTUPDATE,2182637,2018-07-16 10:25:44.996667392,2018-01-08 17:21:10,2018-04-10 15:08:12,2018-07-13 19:47:00,2018-10-23 06:57:21,2019-01-16 18:27:21,NaN


__Three constant columns can be seen in Datasource, Basin, Tenderlot and Suppressed__

- They will ned to be removed as they cannot provide insight into the problem domain.

In [12]:
df.select_dtypes(['category']).describe().T

,count,unique,top,freq
DATASOURCE,2182637,1,DB,2182637
TRIPID,2182637,658964,7327038,19
LINEID,2182637,130,46A,76728
ROUTEID,2182637,588,46A_74,37182
DIRECTION,2182637,2,2,1100273
BASIN,2182637,1,BasDef,2182637
TENDERLOT,0,0,NaN,NaN
SUPPRESSED,4333.0,1.0,0.0,4333.0
JUSTIFICATIONID,4330.0,3526.0,203708.0,19.0
NOTE,2182637,46690,",2703274,",244


## Check for Missing Values

- Thankfully, the majority of missing values are in the columns with constant values and therefore do not affect the features that could provide insight into the problem domain.

In [13]:
df.isnull().sum().sort_values(ascending=False)/len(df)*100

TENDERLOT          100.000000
JUSTIFICATIONID     99.801616
SUPPRESSED          99.801479
ACTUALTIME_DEP       7.539091
ACTUALTIME_ARR       6.286295
DATASOURCE           0.000000
DAYOFSERVICE         0.000000
TRIPID               0.000000
LINEID               0.000000
ROUTEID              0.000000
DIRECTION            0.000000
PLANNEDTIME_ARR      0.000000
PLANNEDTIME_DEP      0.000000
BASIN                0.000000
LASTUPDATE           0.000000
NOTE                 0.000000
dtype: float64

In [14]:
df.head()

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,2018-02-07,6253783,68,68_80,1,87245.0,84600.0,87524.0,84600.0,BasDef,NaN,NaN,NaN,2018-02-28 12:05:11,",2967409,"
1,DB,2018-02-07,6262138,25B,25B_271,2,30517.0,26460.0,32752.0,NaN,BasDef,NaN,NaN,NaN,2018-02-28 12:05:11,",2580260,"
2,DB,2018-02-07,6254942,45A,45A_70,2,35512.0,32100.0,36329.0,32082.0,BasDef,NaN,NaN,NaN,2018-02-28 12:05:11,",2448968,"
3,DB,2018-02-07,6259460,25A,25A_273,1,57261.0,54420.0,58463.0,54443.0,BasDef,NaN,NaN,NaN,2018-02-28 12:05:11,",3094242,"
4,DB,2018-02-07,6253175,14,14_15,1,85383.0,81600.0,84682.0,81608.0,BasDef,NaN,NaN,NaN,2018-02-28 12:05:11,",2526331,"


In [15]:
continuous_columns = df.select_dtypes([ 'float32']).columns
df[continuous_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
PLANNEDTIME_ARR,2182637.0,55078.746094,17424.822266,18203.0,39683.0,55276.0,69006.0,91680.0
PLANNEDTIME_DEP,2182637.0,51533.277344,17571.636719,16200.0,36000.0,51600.0,65160.0,87600.0
ACTUALTIME_ARR,2045430.0,55062.398438,17338.699219,17546.0,39668.0,55189.0,69019.0,94378.0
ACTUALTIME_DEP,2018086.0,51706.125000,17526.949219,15974.0,36518.0,51618.0,65398.0,97177.0


In [16]:
categorical_columns = df.select_dtypes(['category']).columns
df[categorical_columns].describe().T

,count,unique,top,freq
DATASOURCE,2182637,1,DB,2182637
TRIPID,2182637,658964,7327038,19
LINEID,2182637,130,46A,76728
ROUTEID,2182637,588,46A_74,37182
DIRECTION,2182637,2,2,1100273
BASIN,2182637,1,BasDef,2182637
TENDERLOT,0,0,NaN,NaN
SUPPRESSED,4333.0,1.0,0.0,4333.0
JUSTIFICATIONID,4330.0,3526.0,203708.0,19.0
NOTE,2182637,46690,",2703274,",244


In [17]:
df

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,2018-02-07,6253783,68,68_80,1,87245.0,84600.0,87524.0,84600.0,BasDef,NaN,NaN,NaN,2018-02-28 12:05:11,",2967409,"
1,DB,2018-02-07,6262138,25B,25B_271,2,30517.0,26460.0,32752.0,NaN,BasDef,NaN,NaN,NaN,2018-02-28 12:05:11,",2580260,"
2,DB,2018-02-07,6254942,45A,45A_70,2,35512.0,32100.0,36329.0,32082.0,BasDef,NaN,NaN,NaN,2018-02-28 12:05:11,",2448968,"
3,DB,2018-02-07,6259460,25A,25A_273,1,57261.0,54420.0,58463.0,54443.0,BasDef,NaN,NaN,NaN,2018-02-28 12:05:11,",3094242,"
4,DB,2018-02-07,6253175,14,14_15,1,85383.0,81600.0,84682.0,81608.0,BasDef,NaN,NaN,NaN,2018-02-28 12:05:11,",2526331,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182632,DB,2018-05-14,6765849,123,123_36,2,61560.0,57840.0,61365.0,57859.0,BasDef,NaN,NaN,NaN,2018-06-26 09:13:13,",3216350,"
2182633,DB,2018-05-14,6765469,75,75_17,1,53416.0,48600.0,NaN,48823.0,BasDef,NaN,NaN,NaN,2018-06-26 09:13:13,",2865284,"
2182634,DB,2018-05-14,6765486,33D,33D_62,2,29460.0,26400.0,29904.0,NaN,BasDef,NaN,NaN,NaN,2018-06-26 09:13:13,",3077688,"
2182635,DB,2018-05-14,6764987,70,70_60,1,65277.0,60600.0,66341.0,NaN,BasDef,NaN,NaN,NaN,2018-06-26 09:13:13,",3208841,"


# Data Quality Plan


- DATASOURCE - constant - delete 
- DAYOFSERVICE - converted to datetime 
- TRIPID - convert to category 
- LINEID categorical 
- STOPPOINTID - convert to category
- PLANNEDTIME_ARR PLANNEDTIME_DEP ACTUALTIME_ARR ACTUALTIME_DEP VEHICLEID - category 
- DIRECTION - categorical IB inbound = north/east, OB= outbound/south/wedt
- TENDERLOT - missing vals - delete 
- BASIN - missing vals - delete 
- SUPRESSED - missing vals - delete
- JUSTIFICATION - missing vals - delete 
- LASTUPDATE  - delete 
- NOTE - not enough information to use - delete



In [18]:
del df['NOTE']
del df['LASTUPDATE']
del df['JUSTIFICATIONID']
del df['SUPPRESSED']
del df['BASIN']
del df['TENDERLOT']
del df['DATASOURCE']


In [19]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
0,2018-02-07,6253783,68,68_80,1,87245.0,84600.0,87524.0,84600.0
1,2018-02-07,6262138,25B,25B_271,2,30517.0,26460.0,32752.0,NaN
2,2018-02-07,6254942,45A,45A_70,2,35512.0,32100.0,36329.0,32082.0
3,2018-02-07,6259460,25A,25A_273,1,57261.0,54420.0,58463.0,54443.0
4,2018-02-07,6253175,14,14_15,1,85383.0,81600.0,84682.0,81608.0
...,...,...,...,...,...,...,...,...,...
2182632,2018-05-14,6765849,123,123_36,2,61560.0,57840.0,61365.0,57859.0
2182633,2018-05-14,6765469,75,75_17,1,53416.0,48600.0,NaN,48823.0
2182634,2018-05-14,6765486,33D,33D_62,2,29460.0,26400.0,29904.0,NaN
2182635,2018-05-14,6764987,70,70_60,1,65277.0,60600.0,66341.0,NaN


In [27]:
df['JourneyLengthMINS']= (df['ACTUALTIME_ARR']-df['ACTUALTIME_DEP'])/60

In [32]:
df['PlannedLengthMINS']= (df['PLANNEDTIME_ARR']-df['PLANNEDTIME_DEP'])/60

In [33]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,JourneyLengthMINS,PlannedLengthMINS
0,2018-02-07,6253783,68,68_80,1,87245.0,84600.0,87524.0,84600.0,48.733334,44.083332
1,2018-02-07,6262138,25B,25B_271,2,30517.0,26460.0,32752.0,NaN,NaN,67.616669
2,2018-02-07,6254942,45A,45A_70,2,35512.0,32100.0,36329.0,32082.0,70.783333,56.866665
3,2018-02-07,6259460,25A,25A_273,1,57261.0,54420.0,58463.0,54443.0,67.000000,47.349998
4,2018-02-07,6253175,14,14_15,1,85383.0,81600.0,84682.0,81608.0,51.233334,63.049999
...,...,...,...,...,...,...,...,...,...,...,...
2182632,2018-05-14,6765849,123,123_36,2,61560.0,57840.0,61365.0,57859.0,58.433334,62.000000
2182633,2018-05-14,6765469,75,75_17,1,53416.0,48600.0,NaN,48823.0,NaN,80.266670
2182634,2018-05-14,6765486,33D,33D_62,2,29460.0,26400.0,29904.0,NaN,NaN,51.000000
2182635,2018-05-14,6764987,70,70_60,1,65277.0,60600.0,66341.0,NaN,NaN,77.949997


In [20]:
List_Routes = list(df['LINEID'].unique())
len(List_Routes)

130

In [21]:
# df.to_csv("Cleaned_Trips_2018.csv", index=False)

In [22]:
# import sqlite3
# conn = sqlite3.connect("leave_times_2018.sqlite")
# df.to_sql(name='Cleaned_Trips_final', con=conn)

In [23]:
# p2 = pd.read_sql('select * from Cleaned_Trips_final', conn)
# p2

In [50]:
df['JourneyTimeDiff']= (df['JourneyLengthMINS']-df['PlannedLengthMINS'])


In [52]:
df['EarlyOrLate']=  ['Early' if x < 0 else "Late" for x in df['JourneyTimeDiff']]

In [53]:
df.loc[df['LINEID'] == "46A"]

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,JourneyLengthMINS,PlannedLengthMINS,JourneyTimeDiff,EarlyOrLate
15,2018-02-07,6253924,46A,46A_74,1,85.816666,80.133331,5.683334,Late
20,2018-02-07,6262164,46A,46A_74,1,82.666664,73.283333,9.383331,Late
88,2018-02-18,6265839,46A,46A_67,2,67.866669,58.483334,9.383335,Late
89,2018-02-18,6279639,46A,46A_67,2,68.083336,58.483334,9.600002,Late
90,2018-02-18,6276477,46A,46A_67,2,74.133331,58.483334,15.649998,Late
...,...,...,...,...,...,...,...,...,...
2182408,2018-05-15,6756827,46A,46A_67,2,62.000000,54.483334,7.516666,Late
2182413,2018-05-15,6756837,46A,46A_67,2,53.783333,59.583332,-5.799999,Early
2182458,2018-05-15,6764906,46A,46A_74,1,68.699997,74.633331,-5.933334,Early
2182470,2018-05-15,6762038,46A,46A_74,1,96.633331,83.833336,12.799995,Late


In [54]:
df['EarlyOrLate'].value_counts(normalize=True) * 100

Late     77.589173
Early    22.410827
Name: EarlyOrLate, dtype: float64